In [1]:
import glob
import pickle
import numpy as np
!pip install music21
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.callbacks import History 

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
    100% |████████████████████████████████| 17.9MB 1.8MB/s 
  Running setup.py bdist_wheel for music21 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/66/0d/71/ed677f5bec1eca6f924423a317eeb7e4df4859353861d58d44
Successfully built music21


Using TensorFlow backend.


In [2]:
!pip3 install unidecode
import unidecode
import string
import random
import re
import numpy as np
all_characters = string.printable
n_characters = len(all_characters)

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# stuff needed for colaboratory
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# mount drive
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
notes_corpus = []

for file in glob.glob("drive/midi/*.mid"):
    
    print("Extracting MIDI File: ", file)
    midi_stream = converter.parse(file)

    notes = None

    partition = instrument.partitionByInstrument(midi_stream)

    if partition: 
        notes = partition.parts[0].recurse()
    else: 
        notes = midi_stream.flat.notes

    for element in notes:
        if isinstance(element, note.Note):
            notes_corpus.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes_corpus.append('.'.join(str(n) for n in element.normalOrder))           

with open('drive/data/notes_corpus', 'wb') as filepath:
    pickle.dump(notes_corpus, filepath)


Extracting MIDI File:  drive/midi/Zelda_Overworld.mid
Extracting MIDI File:  drive/midi/z_aeristhemepiano.mid
Extracting MIDI File:  drive/midi/VincentPiano.mid
Extracting MIDI File:  drive/midi/ViviinAlexandria.mid
Extracting MIDI File:  drive/midi/waltz_de_choco.mid
Extracting MIDI File:  drive/midi/thenightmarebegins.mid
Extracting MIDI File:  drive/midi/thoughts.mid
Extracting MIDI File:  drive/midi/traitor.mid
Extracting MIDI File:  drive/midi/tpirtsd-piano.mid
Extracting MIDI File:  drive/midi/ultros.mid
Extracting MIDI File:  drive/midi/ultimafro.mid
Extracting MIDI File:  drive/midi/tifap.mid
Extracting MIDI File:  drive/midi/relmstheme-piano.mid
Extracting MIDI File:  drive/midi/Suteki_Da_Ne_(Piano_Version).mid
Extracting MIDI File:  drive/midi/Rydia_pc.mid
Extracting MIDI File:  drive/midi/Still_Alive-1.mid
Extracting MIDI File:  drive/midi/roseofmay-piano.mid
Extracting MIDI File:  drive/midi/rufus.mid
Extracting MIDI File:  drive/midi/sera_.mid
Extracting MIDI File:  drive/

Extracting MIDI File:  drive/midi/Ff7-Cinco.mid
Extracting MIDI File:  drive/midi/FF6epitaph_piano.mid
Extracting MIDI File:  drive/midi/FF4.mid
Extracting MIDI File:  drive/midi/ff4-fight1.mid
Extracting MIDI File:  drive/midi/ff4-town.mid
Extracting MIDI File:  drive/midi/ff4pclov.mid
Extracting MIDI File:  drive/midi/ff4-airship.mid
Extracting MIDI File:  drive/midi/FF3_Third_Phase_Final_(Piano).mid
Extracting MIDI File:  drive/midi/FF3_Battle_(Piano).mid
Extracting MIDI File:  drive/midi/Ff4-BattleLust.mid
Extracting MIDI File:  drive/midi/ff4_piano_collections-main_theme.mid
Extracting MIDI File:  drive/midi/Eternal_Harvest.mid
Extracting MIDI File:  drive/midi/costadsol.mid
Extracting MIDI File:  drive/midi/EyesOnMePiano.mid
Extracting MIDI File:  drive/midi/ff1battp.mid
Extracting MIDI File:  drive/midi/electric_de_chocobo.mid
Extracting MIDI File:  drive/midi/decisive.mid
Extracting MIDI File:  drive/midi/dontbeafraid.mid
Extracting MIDI File:  drive/midi/dayafter.mid
Extractin

In [0]:
with open('drive/data/cleaned_corpus', 'rb') as filepath:
    notes_corpus = pickle.load(filepath)

In [5]:
vocab_size = len(set(notes_corpus))
vocab_size

358

In [0]:
window_size = 60
note_sequence_input = []
next_note_output = []

notes = sorted(set(notes_corpus))
note2int = dict((note, num) for num, note in enumerate(notes))

for i in range(0, len(notes_corpus) - window_size):
    current_sequence = [note2int[note] for note in notes_corpus[i:window_size+i]]
    next_note = note2int[notes_corpus[window_size+i]]
    note_sequence_input.append(current_sequence)
    next_note_output.append(next_note)

In [7]:
training_data = np.reshape(note_sequence_input, (len(note_sequence_input), window_size, 1))
training_data = training_data / float(vocab_size)
training_data.shape

(64525, 60, 1)

In [8]:
training_label = np_utils.to_categorical(next_note_output)
training_label.shape

(64525, 358)

In [0]:
model = Sequential()
model.add(LSTM(500, input_shape=(training_data.shape[1], training_data.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(500, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(500))
model.add(Dense(400))
model.add(Dropout(0.3))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
filepath = "drive/data/model-weights-bigger.hdf5"
model_checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [model_checkpoint]

In [0]:
model.fit(training_data, training_label, epochs=200, batch_size=64, callbacks=callbacks_list)

Epoch 1/200
64525/64525 [==============================] - 452s 7ms/step - loss: 0.1368
Epoch 2/200
 6080/64525 [=>............................] - ETA: 6:51 - loss: 0.1094

64525/64525 [==============================] - 458s 7ms/step - loss: 0.1342
Epoch 3/200
14080/64525 [=====>........................] - ETA: 6:02 - loss: 0.1275

64525/64525 [==============================] - 461s 7ms/step - loss: 0.1331
Epoch 4/200
  640/64525 [..............................] - ETA: 7:27 - loss: 0.0925

In [0]:
model.load_weights('drive/data/model-weights-bigger.hdf5')

In [0]:
starting_sequence = np.random.randint(219, size=window_size)
pattern_sequence = starting_sequence.tolist()
prediction_output = []

int2note = dict((num, note) for num, note in enumerate(notes))

for i in range(900):
    prediction_input = numpy.reshape(pattern_sequence, (1, len(pattern_sequence), 1))
    prediction_input = prediction_input / float(vocab_size)

    if i%3 == 0:   
        print('\r', 'Predicting.  Note: ', i, end='')  
    if i%3 == 1:   
        print('\r', 'Predicting.. Note: ', i, end='')
    if i%3 == 2:   
        print('\r', 'Predicting...Note: ', i, end='')
    prediction = model.predict(prediction_input, verbose=0)

    index = numpy.argmax(prediction)
    note_instance = int2note[index]
    prediction_output.append(note_instance)

    pattern_sequence.append(index)
    pattern_sequence = pattern_sequence[1:len(pattern_sequence)]    
  
prediction_output = prediction_output[300:len(prediction_output)]

 Predicting...Note:  899

In [0]:
offset = 0
output_notes = []

for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        chord_array = pattern.split('.')
        chord_notes = []
        for note_instance in chord_array:
            note_object = note.Note(int(note_instance))
            note_object.storedInstrument = instrument.Piano()
            chord_notes.append(note_object)
        chord_object = chord.Chord(chord_notes)
        chord_object.offset = offset
        output_notes.append(chord_object)
    else:
        note_object = note.Note(pattern)
        note_object.offset = offset
        note_object.storedInstrument = instrument.Piano()
        output_notes.append(note_object)

    offset += 0.75

midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='drive/midi/test_output.mid')

'test_output.mid'

In [0]:
!pkill -9 -f ipykernel_launcher